<a href="https://colab.research.google.com/github/tmccoy94/SportsData/blob/main/CreateDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
import pandas as pd
from google.colab import drive
import os

In [2]:
#drive.flush_and_unmount() #uncomment out to fix drive issues
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
os.chdir('/content/drive/MyDrive/SportsDataProject/')

In [4]:
!ls

CreateDB.ipynb	SportsData.db


In [9]:
class SqliteDBManager:
    def __init__(self, db_name):
        """
        Initialize the SportsDBManager with a default database name.
        """
        self.db_name = db_name #'SportsData.db'
        self.conn = None

    def connect(self):
        """Establish a connection to the database if not already connected."""
        if not self.conn:
            self.conn = sqlite3.connect(self.db_name)

    def close(self):
        """Close the database connection if open."""
        if self.conn:
            self.conn.close()
            self.conn = None

    def check_tables(self):
        """Check for tables in the database."""
        self.connect()
        cur = self.conn.cursor()
        stmt = "select name from sqlite_master where type='table'"
        cur.execute(stmt)

        output_all = cur.fetchall()
        for row_all in output_all:
            print(row_all)

        self.close()

    def _execute_query(self, query: str, params: tuple = None, commit: bool = False) -> None:
        """
        Execute a database query with error handling.

        Args:
            query (str): The SQL query to execute.
            params (tuple, optional): Parameters to include in the query.
            commit (bool): Whether to commit the transaction (useful for INSERT, UPDATE, DELETE).
        """
        try:
            self.connect()
            cursor = self.conn.cursor()
            if params:
                cursor.execute(query, params)
            else:
                cursor.execute(query)
            if commit:
                self.conn.commit()
        except sqlite3.Error as e:
            print(f"Database error: {e}")
        finally:
            self.close()
    def dataframe_query(self, query: str) -> pd.DataFrame:
        """
        Execute a SQL query and return the result as a pandas DataFrame.
        Args:
            query (str): The SQL query to execute.
            Returns:
            pd.DataFrame: The result of the query as a pandas DataFrame.
        """
        try:
            self.connect()
            df = pd.read_sql_query(query, self.conn)
            return df
        except sqlite3.Error as e:
            print(f"Database error: {e}")
        finally:
            self.close()

    def create_table(self, table_name: str, columns: list[tuple]) -> None:
        """
        Create a table with the specified name and columns.

        Args:
            table_name (str): Name of the table.
            columns (list of tuples): Each tuple should contain (column name, data type, primary_key (optional)).
        """
        try:
          # Build the column definitions from the list of tuples
          column_defs = []
          for col in columns:
              col_def = f"{col[0]} {col[1]}"
              if len(col) == 3 and col[2].upper() == "PRIMARY KEY":
                  col_def += " PRIMARY KEY"
              column_defs.append(col_def)

          create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({', '.join(column_defs)})"

          # Execute the create table query
          self._execute_query(create_table_query, commit=True)
          print(f"Table '{table_name}' created successfully.")
        except Exception:
            print(f"Check self.ctdocs for help")

    def insert_table_records(self, table_name: str, records: list[tuple] = [], debug: bool = False) -> None:
      """
      Insert multiple records into a table.

      Args:
          table_name (str): Name of the table.
          records (list of tuples): Each tuple represents a record to be inserted.
          debug (bool): If True, print debug information.
      """
      # Check all basic issues with function inputs
      if not records:
        raise TypeError("records must be a non-empty list")
      if not isinstance(records[0], tuple):
        raise TypeError("records must be a list of tuples")
      if not isinstance(table_name, str):
        raise TypeError("table_name must be a string")
      if not isinstance(debug, bool):
        raise TypeError("debug must be a boolean")

      try:
        # connect to db
        self.connect()
        cur = self.conn.cursor()
        # get column names of table
        cur.execute(f"PRAGMA table_info({table_name})")
        columns = [row[1] for row in cur.fetchall()]
        # check if number of columns in table matches number of columns in records
        if len(columns) != len(records[0]):
          raise ValueError("Number of columns in table and length of the first record tuple do not match")
        column_names = f"{', '.join(columns)}"
        marks = ', '.join(['?'] * len(columns))
        # write query
        insert_query = f"""INSERT INTO {table_name} ({column_names})
        VALUES ({marks})
        """
        if debug:
          print(f"""insert query: {insert_query}
          1st record: {records[0]}
          """)
          return ("Run without debug to commit")
        # execute & commit
        cur.executemany(insert_query,records)
        self.conn.commit()
        print("Records inserted successfully")
      except sqlite3.Error as e:
        print(f"Database error: {e}")
      except TypeError as e:
        print(f"Type error test: {e}")
      finally:
        self.close()


In [6]:
manager = SqliteDBManager("SportsData.db")

In [7]:
manager.check_tables()

('LEAGUE',)


In [8]:
records = [('serial', 'us', 'nfl', 'nfl_tr', 'american football')]
manager.insert_table_records('LEAGUE',records, debug=True)

ValueError: Number of columns in table and records do not match

In [ ]:
manager.dataframe_query("select * from LEAGUE")

,SERIAL,REGION,NAME,TR_URL_NAME,ODDS_URL_NAME,SPORT
